In [4]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import pandas as pd
URM_path = "data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",", 
                                # header=None, 
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

In [5]:
import scipy.sparse as sps
import numpy as np

mapped_id, original_id = pd.factorize(URM_all_dataframe["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

URM_all_dataframe["UserID"] = URM_all_dataframe["UserID"].map(user_original_ID_to_index)
URM_all_dataframe["ItemID"] = URM_all_dataframe["ItemID"].map(item_original_ID_to_index)

In [6]:
n_users = len(URM_all_dataframe["UserID"].unique())
n_items = len(URM_all_dataframe["ItemID"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)),
                        shape = (n_users, n_items))

URM_all

<12638x22222 sparse matrix of type '<class 'numpy.int64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [8]:
recommender = ItemKNNCFRecommender(URM_train=URM_all)
recommender.load_model("result_experiments/", "ItemKNNCFRecommender_cosine_best_model.zip")

ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender_cosine_best_model.zip'
ItemKNNCFRecommender: Loading complete


In [9]:
from Recommenders.NonPersonalizedRecommender import TopPop
top_recommender = TopPop(URM_all)
top_recommender.fit()

In [11]:
URM_path = "data_target_users_test.csv"
URM_all_submission_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",", 
                                # header=None, 
                                dtype={0:int},
                                engine='python')

URM_all_submission_dataframe.columns = ["UserID"]

In [12]:
URM_all_submission_dataframe["UserIDMapped"] = URM_all_submission_dataframe["UserID"].map(user_original_ID_to_index)
# First, create a mask for non-null 'UserIDMapped' values
mask = ~URM_all_submission_dataframe['UserIDMapped'].isna()
URM_all_submission_dataframe['item_list'] = top_recommender.recommend(URM_all_submission_dataframe['UserIDMapped'], cutoff=10, remove_seen_flag=False)
URM_all_submission_dataframe.loc[mask, 'item_list'] = pd.Series(recommender.recommend(URM_all_submission_dataframe.loc[mask, 'UserIDMapped'].to_numpy().astype(int), cutoff=10, remove_seen_flag=True), index=URM_all_submission_dataframe.index[mask])

In [13]:
item_index_to_original_id = item_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)
URM_all_submission_dataframe['item_list'] = URM_all_submission_dataframe['item_list'].apply(lambda item_indices: [item_index_to_original_id[index] for index in item_indices])

In [14]:
submission = URM_all_submission_dataframe[["UserID","item_list"]]
submission.columns = ["user_id", "item_list"]
submission['item_list'] = submission['item_list'].apply(lambda x: ' '.join(map(str, x))) # remove brakets

# Convert item_list to string format
submission['item_list'] = submission['item_list'].astype(str)

# Save to CSV without quotes and with the desired format
# submission.to_csv("submission.csv", index=False, header=["user_id", "item_list"], quoting=csv.QUOTE_NONE, sep=',')
def write_csv_without_quotes(df, file_path):
    with open(file_path, 'w', newline='') as f:
        f.write("user_id,item_list\n")
        for index, row in df.iterrows():
            f.write(f"{row['user_id']},{row['item_list']}\n")

# Save the DataFrame to CSV without quotes and with the desired format
write_csv_without_quotes(submission, "submission.csv")

/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_21121/2149847291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['item_list'] = submission['item_list'].apply(lambda x: ' '.join(map(str, x))) # remove brakets
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_21121/2149847291.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['item_list'] = submission['item_list'].astype(str)
